In [96]:
import pandas as pd

FILE = "INFLUD23-17-06-2024.csv"

DATASET = pd.read_csv(FILE, sep=';')

C:\Users\kauan\AppData\Local\Temp\ipykernel_512\638461740.py:5: DtypeWarning: Columns (15,18,20,23,53,61,62,63,72,74,79,91,93,109,130,143,175,178) have mixed types. Specify dtype option on import or set low_memory=False.
  DATASET = pd.read_csv(FILE, sep=';')


In [365]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

df = DATASET.copy()


# Excluir valores nulos da coluna CLASSI_FIN
df = df[df["CLASSI_FIN"].notna()] 

# Filtrar as colunas de interesse
df = df[["CLASSI_FIN", "SG_UF_NOT", "DT_NOTIFIC"]]

# Converter UF para um valor numérico
label_encoder = LabelEncoder()
df["SG_UF_NOT"] = label_encoder.fit_transform(df["SG_UF_NOT"])

# Extrair o dia do ano da data de notificação
dof = pd.to_datetime(df['DT_NOTIFIC'], format="%d/%m/%Y").dt.dayofyear
# Normalizar os valores para o intervalo [0, 1]
min_max_scaler = MinMaxScaler()
# df["DOF_NOTIFIC"] = min_max_scaler.fit_transform(dof.values.reshape(-1, 1))
df["DOF_NOTIFIC"] = dof

df.drop(columns=["DT_NOTIFIC"], inplace=True)

df

,CLASSI_FIN,SG_UF_NOT,DOF_NOTIFIC
0,5.0,17,11
1,5.0,23,19
2,4.0,25,23
3,2.0,6,17
5,4.0,6,26
...,...,...,...
278977,4.0,25,340
278978,4.0,8,363
278979,5.0,4,349
278980,4.0,17,355


In [368]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

x = df[['DOF_NOTIFIC', 'SG_UF_NOT']]
y = df['CLASSI_FIN']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LogisticRegression(multi_class="auto", solver="liblinear", max_iter=1000)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

accuracy

c:\Users\kauan\Projects\uni\machine-learning-abp\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


0.5674534069435696

In [387]:
# Calcular a frequência das classificações por dia do ano
freq_class__dof = df.groupby(["CLASSI_FIN", "DOF_NOTIFIC"]).size().reset_index(name="FREQ")
freq_class__dof["FREQ"] = freq_class__dof["FREQ"] / freq_class__dof["FREQ"].max()
freq_class__dof.sort_values(by="FREQ", ascending=False)
# freq_class__dof[(freq_class__dof["DOF_NOTIFIC"] == 152) & (freq_class__dof["CLASSI_FIN"] == 1)]

,CLASSI_FIN,DOF_NOTIFIC,FREQ
1231,4.0,149,1.000000
1224,4.0,142,0.981896
1245,4.0,163,0.974441
1196,4.0,114,0.965921
1238,4.0,156,0.945687
...,...,...,...
20,1.0,21,0.001065
978,3.0,259,0.001065
0,1.0,1,0.001065
1062,3.0,344,0.001065


In [393]:
import numpy as np

DATE = "15/06"
UF = "SP"

prompt = [[
    # min_max_scaler.transform([[pd.to_datetime(DATE, format="%d/%m").dayofyear]])[0][0],
    pd.to_datetime(DATE, format="%d/%m").dayofyear,
    label_encoder.transform([UF])[0]
]]
# print(prompt[0][0])

# model.predict(prompt)
predict = model.predict(prompt)
proba = model.predict_proba(prompt)
freq = freq_class__dof[freq_class__dof["DOF_NOTIFIC"] == 152].sort_values(by="CLASSI_FIN")["FREQ"].values

proba_predict = np.array([])
for classi in range(5):
    proba_ponderada = proba[0][classi] * (1 + freq[classi] / 100)
    proba_predict = np.append(proba_predict, proba_ponderada)

# proba_predict /= proba_predict.sum()

print("Probabilidade de cada classe (doença) para a entrada do usuário:")
for classe, probabilidade in enumerate(proba_predict):
    print(f"Classe {classe + 1}: {probabilidade:.2f}")

Probabilidade de cada classe (doença) para a entrada do usuário:
Classe 1: 0.08
Classe 2: 0.17
Classe 3: 0.01
Classe 4: 0.52
Classe 5: 0.21


c:\Users\kauan\Projects\uni\machine-learning-abp\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\kauan\Projects\uni\machine-learning-abp\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
